# Loading data with CombinedCesnetDataset

### Import

In [1]:
from tqdm import tqdm
import logging

from cesnet_tszoo.utils.enums import AgreggationType, SourceType, TimeFormat, DatasetType
from cesnet_tszoo.datasets import CESNET_TimeSeries24
from cesnet_tszoo.configs import CombinedConfig # Combined dataset MUST use CombinedConfig

### Setting logger

In [2]:
logging.basicConfig(
    level=logging.INFO,
    format="[%(asctime)s][%(name)s][%(levelname)s] - %(message)s")

### Preparing dataset

In [3]:
combined_dataset = CESNET_TimeSeries24.get_dataset(data_root="/some_directory/", source_type=SourceType.INSTITUTION_SUBNETS, aggregation=AgreggationType.AGG_1_HOUR, dataset_type=DatasetType.COMBINED, display_details=True)

[2025-08-17 15:46:25,005][wrapper_dataset][INFO] - Dataset is combined. Use cesnet_tszoo.configs.CombinedConfig



Dataset details:

    AgreggationType.AGG_1_HOUR
        Time indices: range(0, 6717)
        Datetime: (datetime.datetime(2023, 10, 9, 0, 0, tzinfo=datetime.timezone.utc), datetime.datetime(2024, 7, 14, 21, 0, tzinfo=datetime.timezone.utc))

    SourceType.INSTITUTION_SUBNETS
        Time series indices: [0 1 2 3 4 ... 543 544 545 546 547], Length=548; use 'get_available_ts_indices' for full list
        Features with default values: {'n_flows': 0, 'n_packets': 0, 'n_bytes': 0, 'tcp_udp_ratio_packets': 0.5, 'tcp_udp_ratio_bytes': 0.5, 'dir_ratio_packets': 0.5, 'dir_ratio_bytes': 0.5, 'avg_duration': 0, 'avg_ttl': 0, 'sum_n_dest_asn': 0, 'avg_n_dest_asn': 0, 'std_n_dest_asn': 0, 'sum_n_dest_ports': 0, 'avg_n_dest_ports': 0, 'std_n_dest_ports': 0, 'sum_n_dest_ip': 0, 'avg_n_dest_ip': 0, 'std_n_dest_ip': 0}
        
        Additional data: ['ids_relationship', 'weekends_and_holidays']
        


### Loading data with DataLoader

- Load data using Pytorch Dataloader.
- Last batch is never dropped (unless sliding_window is used).
- Workers affect how many processes will be used for loading data for specific set.
    - Workers set to 0, means loading will be ran on main process.
    - Set workers can be overriden in `get_*_dataloader` with parameter `workers`.
- Batch size affect how many times for every time series will be in one batch (differs when sliding window is used).
- Batch consists of: (only when sliding window is not used).
    - When `time_format` is not TimeFormat.DATETIME, then batch is one Numpy array of shape `(train_ts/val_ts/test_ts, batch_size, features_to_take + used ids)`.
    - When `time_format` is TimeFormat.DATETIME, then batch is a tuple: (Numpy array of shape `(train_ts/val_ts/test_ts, batch_size, features_to_take + used ids (without time))`, Numpy array of shape `(batch_size)`)

In [4]:
config = CombinedConfig(train_ts=0.5, val_ts=0.2, test_ts=0.1, train_time_period=0.5, val_time_period=0.3, test_time_period=0.2, features_to_take="all", time_format=TimeFormat.ID_TIME,
                         train_workers=0, val_workers=0, test_workers=0, init_workers=0,
                         train_batch_size=32, val_batch_size=64, test_batch_size=128)
combined_dataset.set_dataset_config_and_initialize(config, display_config_details=True, workers=0)

[2025-08-17 15:46:25,011][combined_config][INFO] - Quick validation succeeded.
[2025-08-17 15:46:25,030][combined_config][INFO] - Finalization and validation completed successfully.
[2025-08-17 15:46:25,034][cesnet_dataset][INFO] - Updating config for train set.
100%|██████████| 274/274 [00:00<00:00, 1376.10it/s]
[2025-08-17 15:46:25,251][cesnet_dataset][INFO] - Updating config for val set.
100%|██████████| 109/109 [00:00<00:00, 1296.25it/s]
[2025-08-17 15:46:25,347][cesnet_dataset][INFO] - Updating config for test set.
100%|██████████| 54/54 [00:00<00:00, 1212.03it/s]
[2025-08-17 15:46:25,398][cesnet_dataset][INFO] - Config initialized successfully.



Config Details
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_1_HOUR
    Source: SourceType.INSTITUTION_SUBNETS

    Time series
        Train time series IDs: [254 283 494 546 458 ... 495 380 236  85 120], Length=274
        Val time series IDs: [214 418 409  23 278 ...  26  21 375 524 490], Length=109
        Test time series IDs: [ 70  94 358  57  99 ... 454 137 134 291 364], Length=54
    Time periods
        Train time periods: range(0, 3359)
        Val time periods: range(3359, 5374)
        Test time periods: range(5374, 6717)
    Features
        Taken features: ['n_flows', 'n_packets', 'n_bytes', 'sum_n_dest_asn', 'avg_n_dest_asn', 'std_n_dest_asn', 'sum_n_dest_ports', 'avg_n_dest_ports', 'std_n_dest_ports', 'sum_n_dest_ip', 'avg_n_dest_ip', 'std_n_dest_ip', 'tcp_udp_ratio_packets', 'tcp_udp_ratio_bytes', 'dir_ratio_packets', 'dir_ratio_bytes', 'avg_duration', 'avg_ttl']
        Default values: [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 

You can also change set batch sizes later with `update_dataset_config_and_initialize` or `set_batch_sizes`.

In [5]:
combined_dataset.update_dataset_config_and_initialize(train_batch_size=33, val_batch_size=65, test_batch_size="config")
# Or
combined_dataset.set_batch_sizes(train_batch_size=33, val_batch_size=65, test_batch_size="config")

[2025-08-17 15:46:25,403][cesnet_dataset][INFO] - Re-initialization is not needed.
[2025-08-17 15:46:25,404][cesnet_dataset][INFO] - Configuration has been changed successfuly.
[2025-08-17 15:46:25,405][cesnet_dataset][INFO] - Re-initialization is not needed.
[2025-08-17 15:46:25,406][cesnet_dataset][INFO] - Configuration has been changed successfuly.
[2025-08-17 15:46:25,406][cesnet_dataset][INFO] - Batch sizes has been changed successfuly.


You can also change set workers later with `update_dataset_config_and_initialize` or `set_workers`.

In [6]:
combined_dataset.update_dataset_config_and_initialize(train_workers=0, val_workers=0, test_workers=0, init_workers=0)
# Or
combined_dataset.set_workers(train_workers=0, val_workers=0, test_workers=0, init_workers=0)

[2025-08-17 15:46:25,411][cesnet_dataset][INFO] - Re-initialization is not needed.
[2025-08-17 15:46:25,412][cesnet_dataset][INFO] - Configuration has been changed successfuly.
[2025-08-17 15:46:25,413][cesnet_dataset][INFO] - Re-initialization is not needed.
[2025-08-17 15:46:25,413][cesnet_dataset][INFO] - Configuration has been changed successfuly.
[2025-08-17 15:46:25,413][cesnet_dataset][INFO] - Workers has been changed successfuly.


#### Train set

- Affected by `train_batch_size`.
- Affected by `train_workers`.

In [7]:
dataloader = combined_dataset.get_train_dataloader(workers="config")

batches = []

for batch in tqdm(dataloader):
    batches.append(batch)
    
display(batches[0].shape)

[2025-08-17 15:46:25,423][cesnet_dataset][INFO] - Created new cached train_dataloader.
100%|██████████| 105/105 [00:02<00:00, 47.05it/s]


(274, 32, 20)

#### Val set

- Affected by `val_batch_size`.
- Affected by `val_workers`.

In [8]:
dataloader = combined_dataset.get_val_dataloader(workers="config")

batches = []

for batch in tqdm(dataloader):
    batches.append(batch)
    
display(batches[0].shape)

[2025-08-17 15:46:27,664][cesnet_dataset][INFO] - Created new cached val_dataloader.
100%|██████████| 32/32 [00:00<00:00, 68.96it/s]


(109, 64, 20)

#### Test set

- Affected by `test_batch_size`.
- Affected by `test_workers`.

In [9]:
dataloader = combined_dataset.get_test_dataloader(workers="config")

batches = []

for batch in tqdm(dataloader):
    batches.append(batch)
    
display(batches[0].shape)

[2025-08-17 15:46:28,144][cesnet_dataset][INFO] - Created new cached test_dataloader.
100%|██████████| 11/11 [00:00<00:00, 108.71it/s]


(54, 128, 20)

#### Using time_format=TimeFormat.DATETIME

In [10]:
config = CombinedConfig(train_ts=0.5, val_ts=0.2, test_ts=0.1, train_time_period=0.5, val_time_period=0.3, test_time_period=0.2, features_to_take="all", time_format=TimeFormat.DATETIME,
                         train_workers=0, val_workers=0, test_workers=0, init_workers=0,
                         train_batch_size=32, val_batch_size=64, test_batch_size=128)
combined_dataset.set_dataset_config_and_initialize(config, display_config_details=True, workers=0)

[2025-08-17 15:46:28,253][combined_config][INFO] - Quick validation succeeded.
[2025-08-17 15:46:28,276][combined_config][INFO] - Finalization and validation completed successfully.
[2025-08-17 15:46:28,280][cesnet_dataset][INFO] - Updating config for train set.
100%|██████████| 274/274 [00:00<00:00, 2675.26it/s]
[2025-08-17 15:46:28,388][cesnet_dataset][INFO] - Updating config for val set.
100%|██████████| 109/109 [00:00<00:00, 3111.42it/s]
[2025-08-17 15:46:28,428][cesnet_dataset][INFO] - Updating config for test set.
100%|██████████| 54/54 [00:00<00:00, 3270.79it/s]
[2025-08-17 15:46:28,445][cesnet_dataset][INFO] - Config initialized successfully.



Config Details
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_1_HOUR
    Source: SourceType.INSTITUTION_SUBNETS

    Time series
        Train time series IDs: [531 262 451 258 288 ...  18 493  15 347 229], Length=274
        Val time series IDs: [325 136 429 270   9 ...  11 107 299 512  17], Length=109
        Test time series IDs: [220 119 213 538 329 ...  90 335 381 432  57], Length=54
    Time periods
        Train time periods: range(0, 3359)
        Val time periods: range(3359, 5374)
        Test time periods: range(5374, 6717)
    Features
        Taken features: ['n_flows', 'n_packets', 'n_bytes', 'sum_n_dest_asn', 'avg_n_dest_asn', 'std_n_dest_asn', 'sum_n_dest_ports', 'avg_n_dest_ports', 'std_n_dest_ports', 'sum_n_dest_ip', 'avg_n_dest_ip', 'std_n_dest_ip', 'tcp_udp_ratio_packets', 'tcp_udp_ratio_bytes', 'dir_ratio_packets', 'dir_ratio_bytes', 'avg_duration', 'avg_ttl']
        Default values: [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 

In [11]:
dataloader = combined_dataset.get_train_dataloader(workers="config")

batches = []

for batch in tqdm(dataloader):
    batches.append(batch)
    
display(batches[0][0].shape) # data without time
display(batches[0][1].shape) # time

[2025-08-17 15:46:28,453][cesnet_dataset][INFO] - Created new cached train_dataloader.
100%|██████████| 105/105 [00:02<00:00, 51.34it/s]


(274, 32, 19)

(32,)

#### Specifying which time series to load

- Every `get_*_dataloader` has parameter `ts_id`.
    - When `ts_id` is None, then it returns as previous examples.
    - When `ts_id` is not None, then it returns only one time series of that specified id.

In [12]:
config = CombinedConfig(train_ts=[177, 176, 319, 267], val_ts=None, test_ts=None, train_time_period=0.5, features_to_take="all", time_format=TimeFormat.ID_TIME,
                         train_workers=0, val_workers=0, test_workers=0, init_workers=0,
                         train_batch_size=32, val_batch_size=64, test_batch_size=128)
combined_dataset.set_dataset_config_and_initialize(config, display_config_details=True, workers=0)

[2025-08-17 15:46:30,507][combined_config][INFO] - Quick validation succeeded.
[2025-08-17 15:46:30,518][combined_config][INFO] - Finalization and validation completed successfully.
[2025-08-17 15:46:30,520][cesnet_dataset][INFO] - Updating config for train set.
100%|██████████| 4/4 [00:00<00:00, 1595.70it/s]
[2025-08-17 15:46:30,525][cesnet_dataset][INFO] - Config initialized successfully.



Config Details
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_1_HOUR
    Source: SourceType.INSTITUTION_SUBNETS

    Time series
        Train time series IDs: [177 176 319 267], Length=4
        Val time series IDs: None
        Test time series IDs: None
    Time periods
        Train time periods: range(0, 3359)
        Val time periods: None
        Test time periods: None
    Features
        Taken features: ['n_flows', 'n_packets', 'n_bytes', 'sum_n_dest_asn', 'avg_n_dest_asn', 'std_n_dest_asn', 'sum_n_dest_ports', 'avg_n_dest_ports', 'std_n_dest_ports', 'sum_n_dest_ip', 'avg_n_dest_ip', 'std_n_dest_ip', 'tcp_udp_ratio_packets', 'tcp_udp_ratio_bytes', 'dir_ratio_packets', 'dir_ratio_bytes', 'avg_duration', 'avg_ttl']
        Default values: [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.5 0.5 0.5 0.5 0.  0. ]
        Time series ID included: True
        Time included: True    
        Time format: TimeFormat.ID_TIME
    Sliding window
      

In [13]:
dataloader = combined_dataset.get_train_dataloader(ts_id=177 , workers="config",)

batches = []

for batch in tqdm(dataloader):
    batches.append(batch)
    
display(batches[0].shape)

[2025-08-17 15:46:30,532][cesnet_dataset][INFO] - Created new cached train_dataloader.
100%|██████████| 105/105 [00:00<00:00, 2008.98it/s]


(1, 32, 20)

#### Sliding window

- Both `sliding_window_size` and `sliding_window_prediction_size` must be set if you want to use sliding window.
- Batch sizes are used for background caching.
- Batch consists of:
    - When `time_format` is not TimeFormat.DATETIME, then batch is a tuple: <br>
    ( <br>
        Numpy array of shape `(train_ts/val_ts/test_ts, sliding_window_size, features_to_take + used ids)`, <br>
        Numpy array of shape `(train_ts/val_ts/test_ts, sliding_window_prediction_size, features_to_take + used ids)` <br>
    ).
    - When `time_format` is TimeFormat.DATETIME, then batch is a tuple: <br>
    ( <br>
        Numpy array of shape `(train_ts/val_ts/test_ts, sliding_window_size, features_to_take + used ids (without time))`, <br>
        Numpy array of shape `(train_ts/val_ts/test_ts, sliding_window_prediction_size, features_to_take + used ids (without time))`, <br>
        Numpy array of shape `(sliding_window_size)` of time, <br>
        Numpy array of shape `(sliding_window_prediction_size)` of time, <br>
    ).
- You can modify sliding window step size with `sliding_window_step`
- You can use `set_shared_size` to set how many times time periods should share.
    - `val_time_period` takes from `train_time_period`
    - `test_time_period` takes from `val_time_period` or `train_time_period`

In [14]:
config = CombinedConfig(train_ts=0.5, val_ts=0.2, test_ts=0.1, train_time_period=range(0, 1000), val_time_period=range(1000, 1500), test_time_period=range(1500, 2000), features_to_take=["n_flows"], time_format=TimeFormat.ID_TIME,
                         train_workers=0, val_workers=0, test_workers=0, init_workers=0,
                         train_batch_size=32, val_batch_size=64, test_batch_size=128,
                         sliding_window_size=22, sliding_window_prediction_size=2, sliding_window_step=2, set_shared_size=0.05)
combined_dataset.set_dataset_config_and_initialize(config, display_config_details=True, workers=0)

[2025-08-17 15:46:30,598][combined_config][INFO] - Quick validation succeeded.
[2025-08-17 15:46:30,616][combined_config][INFO] - Finalization and validation completed successfully.
[2025-08-17 15:46:30,619][cesnet_dataset][INFO] - Updating config for train set.
100%|██████████| 274/274 [00:00<00:00, 8693.32it/s]
[2025-08-17 15:46:30,656][cesnet_dataset][INFO] - Updating config for val set.
100%|██████████| 109/109 [00:00<00:00, 7025.63it/s]
[2025-08-17 15:46:30,677][cesnet_dataset][INFO] - Updating config for test set.
100%|██████████| 54/54 [00:00<00:00, 7201.21it/s]
[2025-08-17 15:46:30,686][cesnet_dataset][INFO] - Config initialized successfully.



Config Details
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_1_HOUR
    Source: SourceType.INSTITUTION_SUBNETS

    Time series
        Train time series IDs: [123 134 460 115 343 ... 199 540 273 242 247], Length=274
        Val time series IDs: [166 435 534 523  66 ... 487 143 226  84 481], Length=109
        Test time series IDs: [346 427 205 391 209 ... 357 422 291 180 408], Length=54
    Time periods
        Train time periods: range(0, 1000)
        Val time periods: range(665, 1500)
        Test time periods: range(1165, 2000)
    Features
        Taken features: ['n_flows']
        Default values: [0.]
        Time series ID included: True
        Time included: True    
        Time format: TimeFormat.ID_TIME
    Sliding window
        Sliding window size: 22
        Sliding window prediction size: 2
        Sliding window step size: 2
    Fillers
        Filler type: None
    Transformers
        Transformer type: None
    Batch sizes
       

In [15]:
dataloader = combined_dataset.get_train_dataloader(workers="config")

batches = []

for sliding_window, sliding_window_prediction in tqdm(dataloader):
    batches.append((sliding_window, sliding_window_prediction))

[2025-08-17 15:46:30,693][cesnet_dataset][INFO] - Created new cached train_dataloader.
100%|██████████| 489/489 [00:00<00:00, 1251.10it/s]


You can also change sliding window parameters later with `update_dataset_config_and_initialize` or `set_sliding_window`.

In [16]:
combined_dataset.update_dataset_config_and_initialize(sliding_window_size=22, sliding_window_prediction_size=3, sliding_window_step="config", set_shared_size="config", workers=0)
# Or
combined_dataset.set_sliding_window(sliding_window_size=22, sliding_window_prediction_size=3, sliding_window_step="config", set_shared_size="config", workers=0)

[2025-08-17 15:46:31,091][cesnet_dataset][INFO] - Re-initialization is not needed.
[2025-08-17 15:46:31,091][combined_config][INFO] - all_batch_size adjusted to 25 as it should be greater than or equal to sliding_window_size + sliding_window_prediction_size.
[2025-08-17 15:46:31,092][cesnet_dataset][INFO] - Destroyed cached train_dataloader.
[2025-08-17 15:46:31,092][cesnet_dataset][INFO] - Configuration has been changed successfuly.
[2025-08-17 15:46:31,093][cesnet_dataset][INFO] - Re-initialization is not needed.
[2025-08-17 15:46:31,093][cesnet_dataset][INFO] - Configuration has been changed successfuly.
[2025-08-17 15:46:31,093][cesnet_dataset][INFO] - Sliding window values has been changed successfuly.


##### Using time_format=TimeFormat.DATETIME

In [17]:
config = CombinedConfig(train_ts=0.5, val_ts=0.2, test_ts=0.1, train_time_period=range(0, 1000), val_time_period=range(978, 1500), test_time_period=range(1478, 2000), features_to_take=["n_flows"], time_format=TimeFormat.DATETIME,
                         train_workers=0, val_workers=0, test_workers=0, init_workers=0,
                         train_batch_size=32, val_batch_size=64, test_batch_size=128,
                         sliding_window_size=22, sliding_window_prediction_size=2, sliding_window_step=2, set_shared_size=100)
combined_dataset.set_dataset_config_and_initialize(config, display_config_details=True, workers=0)

[2025-08-17 15:46:31,099][combined_config][INFO] - Quick validation succeeded.
[2025-08-17 15:46:31,119][combined_config][INFO] - Finalization and validation completed successfully.
[2025-08-17 15:46:31,122][cesnet_dataset][INFO] - Updating config for train set.
100%|██████████| 274/274 [00:00<00:00, 7591.50it/s]
[2025-08-17 15:46:31,162][cesnet_dataset][INFO] - Updating config for val set.
100%|██████████| 109/109 [00:00<00:00, 7260.73it/s]
[2025-08-17 15:46:31,182][cesnet_dataset][INFO] - Updating config for test set.
100%|██████████| 54/54 [00:00<00:00, 7192.52it/s]
[2025-08-17 15:46:31,191][cesnet_dataset][INFO] - Config initialized successfully.



Config Details
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_1_HOUR
    Source: SourceType.INSTITUTION_SUBNETS

    Time series
        Train time series IDs: [448 254 467 505  51 ...  53 235 347  32  34], Length=274
        Val time series IDs: [397 418 456 311  84 ...  47  28 337 216 380], Length=109
        Test time series IDs: [369 291  92 414 245 ... 446 320 300 141 379], Length=54
    Time periods
        Train time periods: range(0, 1000)
        Val time periods: range(900, 1500)
        Test time periods: range(1400, 2000)
    Features
        Taken features: ['n_flows']
        Default values: [0.]
        Time series ID included: True
        Time included: True    
        Time format: TimeFormat.DATETIME
    Sliding window
        Sliding window size: 22
        Sliding window prediction size: 2
        Sliding window step size: 2
    Fillers
        Filler type: None
    Transformers
        Transformer type: None
    Batch sizes
      

In [18]:
dataloader = combined_dataset.get_train_dataloader(workers="config")

batches = []

for sliding_window, sliding_window_prediction, sliding_window_times, sliding_window_prediction_times in tqdm(dataloader):
    batches.append((sliding_window, sliding_window_prediction, sliding_window_times, sliding_window_prediction_times))

[2025-08-17 15:46:31,200][cesnet_dataset][INFO] - Created new cached train_dataloader.
100%|██████████| 489/489 [00:00<00:00, 1301.39it/s]


### Loading data as Dataframe

- Batch size has no effect.
- Sliding window has no effect.
- Returns every time series in `train_ts/val_ts/test_ts` with sets specified time period.
- Data is returned as Pandas Dataframe.
- Workers affect how many processes will be used for loading data for specific set.
    - Workers set to 0, means loading will be ran on main process.
    - Set workers can be overriden in `get_*_df` with parameter `workers`.

In [19]:
config = CombinedConfig(train_ts=0.5, val_ts=0.2, test_ts=0.1, train_time_period=0.5, val_time_period=0.3, test_time_period=0.2, features_to_take="all", time_format=TimeFormat.ID_TIME,
                         train_workers=0, val_workers=0, test_workers=0, init_workers=0)
combined_dataset.set_dataset_config_and_initialize(config, display_config_details=True, workers=0)

[2025-08-17 15:46:31,582][combined_config][INFO] - Quick validation succeeded.
[2025-08-17 15:46:31,600][combined_config][INFO] - Finalization and validation completed successfully.
[2025-08-17 15:46:31,603][cesnet_dataset][INFO] - Updating config for train set.
100%|██████████| 274/274 [00:00<00:00, 2847.03it/s]
[2025-08-17 15:46:31,705][cesnet_dataset][INFO] - Updating config for val set.
100%|██████████| 109/109 [00:00<00:00, 3429.91it/s]
[2025-08-17 15:46:31,741][cesnet_dataset][INFO] - Updating config for test set.
100%|██████████| 54/54 [00:00<00:00, 3854.47it/s]
[2025-08-17 15:46:31,758][cesnet_dataset][INFO] - Config initialized successfully.



Config Details
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_1_HOUR
    Source: SourceType.INSTITUTION_SUBNETS

    Time series
        Train time series IDs: [444 199 133 456 170 ... 137  18 288 186 369], Length=274
        Val time series IDs: [ 70  35 473 276 472 ... 394  21 139 381 434], Length=109
        Test time series IDs: [393 462  54 355 331 ... 455 309 413 543 338], Length=54
    Time periods
        Train time periods: range(0, 3359)
        Val time periods: range(3359, 5374)
        Test time periods: range(5374, 6717)
    Features
        Taken features: ['n_flows', 'n_packets', 'n_bytes', 'sum_n_dest_asn', 'avg_n_dest_asn', 'std_n_dest_asn', 'sum_n_dest_ports', 'avg_n_dest_ports', 'std_n_dest_ports', 'sum_n_dest_ip', 'avg_n_dest_ip', 'std_n_dest_ip', 'tcp_udp_ratio_packets', 'tcp_udp_ratio_bytes', 'dir_ratio_packets', 'dir_ratio_bytes', 'avg_duration', 'avg_ttl']
        Default values: [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 

#### Train set

- Affected by `train_workers`.

In [20]:
df = combined_dataset.get_train_df(as_single_dataframe=True, workers="config")
dfs = combined_dataset.get_train_df(as_single_dataframe=False, workers="config")

df.head(10)

,id_institution_subnet,id_time,n_flows,n_packets,n_bytes,sum_n_dest_asn,avg_n_dest_asn,std_n_dest_asn,sum_n_dest_ports,avg_n_dest_ports,std_n_dest_ports,sum_n_dest_ip,avg_n_dest_ip,std_n_dest_ip,tcp_udp_ratio_packets,tcp_udp_ratio_bytes,dir_ratio_packets,dir_ratio_bytes,avg_duration,avg_ttl
0,444.0,0.0,3117.0,31654.0,14671060.0,714.0,7.44,3.55,780.0,8.12,6.93,1289.0,13.430000,8.480000,0.589844,0.640137,0.469971,0.500000,4.62,144.520004
1,444.0,1.0,3221.0,27277.0,7620848.0,734.0,7.57,3.63,822.0,8.47,7.74,1298.0,13.380000,6.680000,0.540039,0.580078,0.439941,0.469971,4.83,145.669998
2,444.0,2.0,2764.0,21419.0,4903850.0,651.0,7.15,3.37,666.0,7.32,6.39,1136.0,12.480000,7.390000,0.560059,0.580078,0.479980,0.489990,4.94,140.800003
3,444.0,3.0,2657.0,22936.0,6397463.0,680.0,7.16,3.98,736.0,7.75,7.84,1211.0,12.750000,8.900000,0.549805,0.569824,0.509766,0.549805,5.34,140.800003
4,444.0,4.0,3431.0,528278.0,545791793.0,683.0,7.42,7.22,720.0,7.83,9.40,1738.0,18.889999,37.610001,0.560059,0.569824,0.469971,0.500000,5.58,132.690002
5,444.0,5.0,3498.0,145364.0,104372043.0,678.0,7.79,10.26,578.0,6.64,6.49,1888.0,21.700001,49.889999,0.589844,0.600098,0.500000,0.540039,6.79,137.100006
6,444.0,6.0,3441.0,290367.0,254075486.0,763.0,8.29,11.64,699.0,7.60,8.97,1983.0,21.549999,46.799999,0.569824,0.589844,0.489990,0.500000,7.69,141.649994
7,444.0,7.0,3863.0,246564.0,206797985.0,837.0,9.20,11.22,807.0,8.87,10.05,2175.0,23.900000,47.910000,0.560059,0.569824,0.479980,0.520020,8.29,143.559998
8,444.0,8.0,3578.0,150146.0,109354718.0,766.0,7.90,10.61,710.0,7.32,8.34,2063.0,21.270000,43.360001,0.529785,0.549805,0.500000,0.529785,6.45,138.690002
9,444.0,9.0,4187.0,217353.0,145127700.0,869.0,9.45,12.16,817.0,8.88,9.73,2399.0,26.080000,53.380001,0.569824,0.600098,0.479980,0.529785,6.84,148.649994


In [21]:
dfs

[      id_institution_subnet  id_time  n_flows  n_packets      n_bytes  \
 0                     444.0      0.0   3117.0    31654.0   14671060.0   
 1                     444.0      1.0   3221.0    27277.0    7620848.0   
 2                     444.0      2.0   2764.0    21419.0    4903850.0   
 3                     444.0      3.0   2657.0    22936.0    6397463.0   
 4                     444.0      4.0   3431.0   528278.0  545791793.0   
 ...                     ...      ...      ...        ...          ...   
 3354                  444.0   3354.0   2800.0    38100.0   16072577.0   
 3355                  444.0   3355.0   2890.0    51190.0   29091910.0   
 3356                  444.0   3356.0   2821.0    36128.0   14341479.0   
 3357                  444.0   3357.0   3410.0    79019.0   54807953.0   
 3358                  444.0   3358.0   3356.0    65909.0   43130840.0   
 
       sum_n_dest_asn  avg_n_dest_asn  std_n_dest_asn  sum_n_dest_ports  \
 0              714.0            7.

#### Val set

- Affected by `val_workers`.

In [22]:
df = combined_dataset.get_val_df(as_single_dataframe=True, workers="config")
dfs = combined_dataset.get_val_df(as_single_dataframe=False, workers="config")

df.head(10)

,id_institution_subnet,id_time,n_flows,n_packets,n_bytes,sum_n_dest_asn,avg_n_dest_asn,std_n_dest_asn,sum_n_dest_ports,avg_n_dest_ports,std_n_dest_ports,sum_n_dest_ip,avg_n_dest_ip,std_n_dest_ip,tcp_udp_ratio_packets,tcp_udp_ratio_bytes,dir_ratio_packets,dir_ratio_bytes,avg_duration,avg_ttl
0,70.0,3359.0,39061.0,146221.0,13651745.0,15870.0,6.53,4.14,25170.0,10.36,13.94,25883.0,10.65,10.73,0.649902,0.640137,0.489990,0.509766,7.10,110.269997
1,70.0,3360.0,30972.0,87593.0,8218892.0,16100.0,6.57,4.02,17168.0,7.01,4.20,26180.0,10.69,10.13,0.629883,0.620117,0.500000,0.520020,6.32,112.739998
2,70.0,3361.0,30087.0,82719.0,7806338.0,15557.0,6.43,4.16,15876.0,6.57,4.12,25622.0,10.60,10.69,0.609863,0.600098,0.500000,0.529785,6.07,109.570000
3,70.0,3362.0,29431.0,84615.0,11823861.0,15207.0,6.34,4.19,15363.0,6.41,4.03,25096.0,10.47,10.69,0.609863,0.600098,0.509766,0.529785,6.16,109.959999
4,70.0,3363.0,28203.0,76251.0,7029705.0,14495.0,6.05,3.90,14410.0,6.01,3.65,24262.0,10.13,10.44,0.609863,0.600098,0.500000,0.520020,6.39,110.889999
5,70.0,3364.0,21142.0,55652.0,5165673.0,11897.0,5.15,3.25,11619.0,5.03,2.99,18766.0,8.12,8.01,0.629883,0.620117,0.509766,0.540039,6.47,110.400002
6,70.0,3365.0,18390.0,51696.0,4816593.0,10747.0,4.73,3.04,10193.0,4.48,2.70,16509.0,7.26,7.08,0.600098,0.580078,0.520020,0.540039,6.58,109.790001
7,70.0,3366.0,16206.0,41757.0,3879939.0,9720.0,4.45,2.91,8969.0,4.10,2.45,14767.0,6.76,6.35,0.589844,0.569824,0.540039,0.560059,6.42,106.839996
8,70.0,3367.0,16896.0,44590.0,3965048.0,10059.0,4.47,2.90,9326.0,4.14,2.49,15412.0,6.85,6.72,0.589844,0.569824,0.529785,0.549805,6.32,110.199997
9,70.0,3368.0,15971.0,45024.0,4026939.0,9490.0,4.32,2.84,8774.0,3.99,2.34,14521.0,6.61,6.54,0.580078,0.569824,0.529785,0.549805,7.29,110.739998


In [23]:
dfs

[      id_institution_subnet  id_time  n_flows  n_packets     n_bytes  \
 0                      70.0   3359.0  39061.0   146221.0  13651745.0   
 1                      70.0   3360.0  30972.0    87593.0   8218892.0   
 2                      70.0   3361.0  30087.0    82719.0   7806338.0   
 3                      70.0   3362.0  29431.0    84615.0  11823861.0   
 4                      70.0   3363.0  28203.0    76251.0   7029705.0   
 ...                     ...      ...      ...        ...         ...   
 2010                   70.0   5369.0  21064.0    80263.0  18843632.0   
 2011                   70.0   5370.0  21331.0   120441.0  67964043.0   
 2012                   70.0   5371.0  20676.0    71673.0   7288959.0   
 2013                   70.0   5372.0  24097.0    75886.0   7067244.0   
 2014                   70.0   5373.0  23983.0    84193.0  10497360.0   
 
       sum_n_dest_asn  avg_n_dest_asn  std_n_dest_asn  sum_n_dest_ports  \
 0            15870.0            6.53          

#### Test set

- Affected by `test_workers`.

In [24]:
df = combined_dataset.get_test_df(as_single_dataframe=True, workers="config")
dfs = combined_dataset.get_test_df(as_single_dataframe=False, workers="config")

df.head(10)

,id_institution_subnet,id_time,n_flows,n_packets,n_bytes,sum_n_dest_asn,avg_n_dest_asn,std_n_dest_asn,sum_n_dest_ports,avg_n_dest_ports,std_n_dest_ports,sum_n_dest_ip,avg_n_dest_ip,std_n_dest_ip,tcp_udp_ratio_packets,tcp_udp_ratio_bytes,dir_ratio_packets,dir_ratio_bytes,avg_duration,avg_ttl
0,393.0,5374.0,1333.0,10900.0,1905289.0,411.0,8.06,8.480000,549.0,10.76,17.080000,681.0,13.350000,13.790000,0.529785,0.549805,0.459961,0.449951,48.560001,138.350006
1,393.0,5375.0,1346.0,11315.0,2857554.0,421.0,8.77,9.130000,542.0,11.29,18.250000,716.0,14.920000,15.580000,0.449951,0.479980,0.500000,0.509766,54.700001,132.979996
2,393.0,5376.0,1398.0,12333.0,2962100.0,412.0,9.16,9.080000,584.0,12.98,22.120001,718.0,15.960000,17.120001,0.589844,0.609863,0.489990,0.489990,57.270000,140.240005
3,393.0,5377.0,1568.0,14023.0,4246318.0,486.0,9.72,11.430000,534.0,10.68,15.980000,868.0,17.360001,27.680000,0.549805,0.569824,0.479980,0.479980,45.799999,147.690002
4,393.0,5378.0,2321.0,44511.0,33185084.0,505.0,11.48,15.300000,486.0,11.05,16.200001,1286.0,29.230000,52.299999,0.569824,0.589844,0.520020,0.520020,41.349998,148.720001
5,393.0,5379.0,3931.0,166765.0,172524759.0,590.0,12.55,19.950001,440.0,9.36,13.530000,2200.0,46.810001,104.760002,0.600098,0.600098,0.449951,0.419922,38.730000,150.470001
6,393.0,5380.0,3249.0,148143.0,155816407.0,586.0,12.21,18.570000,433.0,9.02,12.200000,1848.0,38.500000,81.739998,0.549805,0.549805,0.469971,0.439941,41.360001,159.009995
7,393.0,5381.0,3931.0,196180.0,194743823.0,657.0,13.14,23.500000,436.0,8.72,12.250000,2234.0,44.680000,103.190002,0.529785,0.540039,0.500000,0.479980,44.799999,147.600006
8,393.0,5382.0,3302.0,74952.0,55395654.0,640.0,14.55,25.780001,403.0,9.16,12.030000,1996.0,45.360001,98.239998,0.529785,0.549805,0.479980,0.469971,60.049999,136.710007
9,393.0,5383.0,3131.0,73784.0,60205962.0,625.0,13.59,23.209999,441.0,9.59,13.050000,1828.0,39.740002,86.510002,0.560059,0.580078,0.489990,0.459961,50.450001,144.330002


In [25]:
dfs

[      id_institution_subnet  id_time  n_flows  n_packets     n_bytes  \
 0                     393.0   5374.0   1333.0    10900.0   1905289.0   
 1                     393.0   5375.0   1346.0    11315.0   2857554.0   
 2                     393.0   5376.0   1398.0    12333.0   2962100.0   
 3                     393.0   5377.0   1568.0    14023.0   4246318.0   
 4                     393.0   5378.0   2321.0    44511.0  33185084.0   
 ...                     ...      ...      ...        ...         ...   
 1338                  393.0   6712.0    913.0    14334.0   2367193.0   
 1339                  393.0   6713.0    865.0    14451.0  11705515.0   
 1340                  393.0   6714.0   1050.0     7617.0   3286874.0   
 1341                  393.0   6715.0    915.0     4086.0    865979.0   
 1342                  393.0   6716.0    936.0     4751.0    766263.0   
 
       sum_n_dest_asn  avg_n_dest_asn  std_n_dest_asn  sum_n_dest_ports  \
 0              411.0            8.06          

### Loading data as singular Numpy array 

- Batch size has no effect.
- Sliding window has no effect.
- Returns every time series in `train_ts/val_ts/test_ts` with sets specified time period.
- Data is returned as one Numpy array.
- Follows similar rules to Dataloader batches, regarding shape (excluding sliding window parameters).
- Workers affect how many processes will be used for loading data for specific set.
    - Workers set to 0, means loading will be ran on main process.
    - Set workers can be overriden in `get_*_numpy` with parameter `workers`.

In [26]:
config = CombinedConfig(train_ts=0.5, val_ts=0.2, test_ts=0.1, train_time_period=0.5, val_time_period=0.3, test_time_period=0.2, features_to_take="all", time_format=TimeFormat.ID_TIME,
                         train_workers=0, val_workers=0, test_workers=0, init_workers=0)
combined_dataset.set_dataset_config_and_initialize(config, display_config_details=True, workers=0)

[2025-08-17 15:46:34,222][combined_config][INFO] - Quick validation succeeded.
[2025-08-17 15:46:34,242][combined_config][INFO] - Finalization and validation completed successfully.
[2025-08-17 15:46:34,246][cesnet_dataset][INFO] - Updating config for train set.
100%|██████████| 274/274 [00:00<00:00, 2837.92it/s]
[2025-08-17 15:46:34,348][cesnet_dataset][INFO] - Updating config for val set.
100%|██████████| 109/109 [00:00<00:00, 3378.88it/s]
[2025-08-17 15:46:34,385][cesnet_dataset][INFO] - Updating config for test set.
100%|██████████| 54/54 [00:00<00:00, 3481.45it/s]
[2025-08-17 15:46:34,402][cesnet_dataset][INFO] - Config initialized successfully.



Config Details
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_1_HOUR
    Source: SourceType.INSTITUTION_SUBNETS

    Time series
        Train time series IDs: [136 438 280  10 100 ... 532 362 185  42  31], Length=274
        Val time series IDs: [533 447   5 401  67 ... 157 443 380 199 334], Length=109
        Test time series IDs: [385 153 221 427 373 ... 245 492 198 210 254], Length=54
    Time periods
        Train time periods: range(0, 3359)
        Val time periods: range(3359, 5374)
        Test time periods: range(5374, 6717)
    Features
        Taken features: ['n_flows', 'n_packets', 'n_bytes', 'sum_n_dest_asn', 'avg_n_dest_asn', 'std_n_dest_asn', 'sum_n_dest_ports', 'avg_n_dest_ports', 'std_n_dest_ports', 'sum_n_dest_ip', 'avg_n_dest_ip', 'std_n_dest_ip', 'tcp_udp_ratio_packets', 'tcp_udp_ratio_bytes', 'dir_ratio_packets', 'dir_ratio_bytes', 'avg_duration', 'avg_ttl']
        Default values: [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 

#### Train set

- Affected by `train_workers`.

In [27]:
numpy_array = combined_dataset.get_train_numpy(workers="config")

display(numpy_array.shape)

(274, 3359, 20)

#### Val set

- Affected by `val_workers`.

In [28]:
numpy_array = combined_dataset.get_val_numpy(workers="config")

display(numpy_array.shape)

(109, 2015, 20)

#### Test set

- Affected by `test_workers`.

In [29]:
numpy_array = combined_dataset.get_test_numpy(workers="config")

display(numpy_array.shape)

(54, 1343, 20)

#### Using time_format=TimeFormat.DATETIME

In [30]:
config = CombinedConfig(train_ts=0.5, val_ts=0.2, test_ts=0.1, train_time_period=0.5, val_time_period=0.3, test_time_period=0.2, features_to_take="all", time_format=TimeFormat.DATETIME,
                         train_workers=0, val_workers=0, test_workers=0, init_workers=0)
combined_dataset.set_dataset_config_and_initialize(config, display_config_details=True, workers=0)

[2025-08-17 15:46:34,767][combined_config][INFO] - Quick validation succeeded.
[2025-08-17 15:46:34,791][combined_config][INFO] - Finalization and validation completed successfully.
[2025-08-17 15:46:34,795][cesnet_dataset][INFO] - Updating config for train set.
100%|██████████| 274/274 [00:00<00:00, 2752.93it/s]
[2025-08-17 15:46:34,899][cesnet_dataset][INFO] - Updating config for val set.
100%|██████████| 109/109 [00:00<00:00, 3100.44it/s]
[2025-08-17 15:46:34,940][cesnet_dataset][INFO] - Updating config for test set.
100%|██████████| 54/54 [00:00<00:00, 3658.71it/s]
[2025-08-17 15:46:34,956][cesnet_dataset][INFO] - Config initialized successfully.



Config Details
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_1_HOUR
    Source: SourceType.INSTITUTION_SUBNETS

    Time series
        Train time series IDs: [541 221  43 370  60 ... 265 219 168 444 127], Length=274
        Val time series IDs: [300 433  14 394 546 ... 151 158 466  22 174], Length=109
        Test time series IDs: [ 35 281 436 262 529 ... 204 343 521 516 150], Length=54
    Time periods
        Train time periods: range(0, 3359)
        Val time periods: range(3359, 5374)
        Test time periods: range(5374, 6717)
    Features
        Taken features: ['n_flows', 'n_packets', 'n_bytes', 'sum_n_dest_asn', 'avg_n_dest_asn', 'std_n_dest_asn', 'sum_n_dest_ports', 'avg_n_dest_ports', 'std_n_dest_ports', 'sum_n_dest_ip', 'avg_n_dest_ip', 'std_n_dest_ip', 'tcp_udp_ratio_packets', 'tcp_udp_ratio_bytes', 'dir_ratio_packets', 'dir_ratio_bytes', 'avg_duration', 'avg_ttl']
        Default values: [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 

In [31]:
numpy_array, times = combined_dataset.get_train_numpy(workers="config")

display(numpy_array.shape)
display(times)

(274, 3359, 19)

array([datetime.datetime(2023, 10, 9, 0, 0, tzinfo=datetime.timezone.utc),
       datetime.datetime(2023, 10, 9, 1, 0, tzinfo=datetime.timezone.utc),
       datetime.datetime(2023, 10, 9, 2, 0, tzinfo=datetime.timezone.utc),
       ...,
       datetime.datetime(2024, 2, 25, 20, 0, tzinfo=datetime.timezone.utc),
       datetime.datetime(2024, 2, 25, 21, 0, tzinfo=datetime.timezone.utc),
       datetime.datetime(2024, 2, 25, 22, 0, tzinfo=datetime.timezone.utc)],
      shape=(3359,), dtype=object)